<a href="https://colab.research.google.com/github/JuliethQP/modelos_ia_prueba/blob/master/modelo_excel_clinicalbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [2]:
!pip install numpy==1.26.4

In [3]:
!pip install --upgrade transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 44.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.3
    Uninstalling transformers-4.52.3:
      Successfully uninstalled transformers-4.52.3


In [4]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset

In [6]:
# 1. Cargar datos
df = pd.read_csv("/content/brain_conditions_detailed_dataset.csv", sep=';')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6056 entries, 0 to 6055
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Case ID        6012 non-null   object
 1   Condition      6056 non-null   object
 2   Age            6056 non-null   int64 
 3   Sex            6056 non-null   object
 4   Clinical Note  6056 non-null   object
 5   Treatment      6056 non-null   object
dtypes: int64(1), object(5)
memory usage: 284.0+ KB


In [7]:
df.head()

,Case ID,Condition,Age,Sex,Clinical Note,Treatment
0,brain_glioma_0001,Brain Glioma,74,F,Patient is experiencing memory lapses and spee...,surgery
1,brain_glioma_0002,Brain Glioma,66,F,Patient is experiencing visual disturbances an...,chemotherapy
2,brain_glioma_0003,Brain Glioma,62,F,Patient is experiencing memory lapses and dizz...,surgery
3,brain_glioma_0004,Brain Glioma,24,M,Patient is experiencing memory lapses and pers...,surgery
4,brain_glioma_0005,Brain Glioma,69,M,Patient is experiencing dizziness and frequent...,surgery


In [8]:
# 2. Preprocesamiento: combinar variables estructuradas
df = df.dropna(subset=["Condition", "Age", "Sex", "Clinical Note", "Treatment"])
df["text"] = "Condition: " + df["Condition"] + "; Age: " + df["Age"].astype(str) + "; Sex: " + df["Sex"] + "; Note: " + df["Clinical Note"]

In [9]:
df["text"].head()

,text
0,Condition: Brain Glioma; Age: 74; Sex: F; Note...
1,Condition: Brain Glioma; Age: 66; Sex: F; Note...
2,Condition: Brain Glioma; Age: 62; Sex: F; Note...
3,Condition: Brain Glioma; Age: 24; Sex: M; Note...
4,Condition: Brain Glioma; Age: 69; Sex: M; Note...


In [10]:
label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["Treatment"])

In [11]:
# 3. Tokenizador
model_name = "medicalai/ClinicalBERT"

tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [12]:
def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=256)

# 4. Crear dataset
dataset = Dataset.from_pandas(df[["text", "label"]])
dataset = dataset.map(tokenize, batched=True)
dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

# 5. División
train_test = dataset.train_test_split(test_size=0.2)
train_dataset = train_test["train"]
eval_dataset = train_test["test"]

# 6. Modelo
num_labels = len(label_encoder.classes_)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

Map:   0%|          | 0/6056 [00:00<?, ? examples/s]

pytorch_model.bin:   0%|          | 0.00/542M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at medicalai/ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
# 7. Argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=1,
    load_best_model_at_end=True,
    report_to="none"
)

In [14]:
# 8. Entrenador
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

In [ ]:
# 9. Entrenamiento
trainer.train()

# 10. Evaluación
predictions = trainer.predict(eval_dataset)
preds = torch.argmax(torch.tensor(predictions.predictions), dim=1)
y_true = predictions.label_ids

print(classification_report(y_true, preds, target_names=label_encoder.classes_))

Epoch,Training Loss,Validation Loss
1,1.199900,1.210202
2,1.224700,1.200989
